# Homework 06 : Numerical Integration

## Objectives

The main objectives of this homework are as follows.

- Learn about markup, in particular $\LaTeX$ in notebooks, for good and elegant documentation.
- Learn more about numerical integration, especially `scipy.integrate.quad`.

## Initialization

As always you should add initialization to the top of your notebook.

In [2]:
import numpy as np
import scipy.integrate as integ
import scipy.special as sf
import matplotlib.pyplot as plt
%matplotlib inline
# Globally fix plot styling
import matplotlib as mpl
mpl.rc('xtick', direction='in', top=True)
mpl.rc('ytick', direction='in', right=True)
mpl.rc('xtick.minor', visible=True)
mpl.rc('ytick.minor', visible=True)

## An aside: some basic $\LaTeX$.

Notebooks, such as this one, provide both code and documentation.  The documentation includes formatting through *Markdown* and equations using $\LaTeX$.  We will become more familiar with some aspects of creating documentation througout the semester.

If you are unfamiliar with $\LaTeX$, it is one of the best systems available for typesetting documents, particularly those involving mathematics, and is a standard tool used in communicating results in the sciences (at least in physics).  It was written over 40 years ago by Donald Knuth in response to electronic proofs of a book he had written.  At the time, publishers were switching from hand typesetting to electronic typesetting of documents.  The proofs were so poorly typeset that he wrote his own system.  This includes tools for creating fonts and a complete set of fonts with all the needed mathematical symbols (something nonexistent at the time and only recently can you find complete sets of fonts which include mathematical symbols).

Here we will get a brief introduction to typesetting simple mathematical expressions.  These can appear in the documentation in the notebook but also in figures.

To typeset a formula we use $\LaTeX$ in markdown cells.  There are many examples of this in notebooks. You can look at the unformatted text in any notebook by double clicking on the cell containing the text. To insert an equation, the dollar sign, `$`, is used to denote where the equation begins and ends.  In notebooks, you should wrap any latex code in dollar characters.

#### A Basic Equation

To get you warmed up try typing in a simple equation, such as $x + y = 3$ but pick your own for fun, surrounded by `$` signs.

###### Your Equation:

```### YOUR SOLUTION HERE```

#### Superscripts and Subscripts

Not every equation is so simple.   To create superscripts and subscripts characters use the caret character `^` and underscore `_`, respectively.  These will only raise the characters immediately after them, unless you wrap subsequent text in curly braces, such as `x^{1234}`.  To get an idea of how this works, try typing some superscripts or subscripts below.  Don't forget to include the `$` signs!

Some examples:

- `x^1` gives $ x^1 $
- `x^12` gives $ x^12 $  (notice what happens if you do not use curly braces)
- `x^{12}` gives $ x^{12} $

###### Your Superscript/Subscript:

```### YOUR SOLUTION HERE```

#### Fractions

Fractions are typeset using `\frac{ numerator }{ denominator }`.  Try typing a fraction below.

###### Fraction Example:

```### YOUR SOLUTION HERE```

## Romberg Integration

Romberg integration of a function can be performed using `scipy.integrate.romberg`.  You should of course look up its documentation.  We will explore its use by considering the integral

$$ \int_0^\phi \sqrt{2-\sin^2 x} \,\mathrm{d}x = \sqrt{2} E\!\left( \phi, \frac12 \right), $$

where $E(\phi,k)$ is the incomplete elliptic integral of the second kind.  It is available in SciPy as `scipy.special.ellipeinc`.

To begin, evaluate this integral using Romberg integration for $\phi=48$.  Print the resulting value, its fractional error, and the number of function evaluations required for the computation. [*Note:* Using `show=True` provides most of the required information.  It just gets printed instead of stored in a variable.  That is sufficient for our purposes.]

In [3]:
### BEGIN SOLUTION
def f(x) :
    return np.sqrt(2-np.sin(x)**2)

res = integ.romberg(f, 0, 48, show=True)

true_value = np.sqrt(2) * sf.ellipeinc(48, 0.5)
print("Integral =", res, " fractional error =", np.abs(1-res/true_value))
### END SOLUTION

Romberg integration of <function vectorize1.<locals>.vfunc at 0x00000259A6FD9B70> from [0, 48]

 Steps  StepSize   Results
     1 48.000000 62.437371 
     2 24.000000 57.288562 55.572292 
     4 12.000000 56.443751 56.162147 56.201471 
     8  6.000000 56.263055 56.202823 56.205534 56.205599 
    16  3.000000 56.218773 56.204012 56.204091 56.204068 56.204062 
    32  1.500000 58.362684 59.077321 59.268875 59.317522 59.329732 59.332787 
    64  0.750000 58.450443 58.479696 58.439854 58.426695 58.423201 58.422315 58.422093 
   128  0.375000 58.465569 58.470611 58.470006 58.470484 58.470656 58.470703 58.470714 58.470717 
   256  0.187500 58.469253 58.470481 58.470472 58.470479 58.470479 58.470479 58.470479 58.470479 58.470479 
   512  0.093750 58.470166 58.470470 58.470469 58.470469 58.470469 58.470469 58.470469 58.470469 58.470469 58.470469 
  1024  0.046875 58.470393 58.470469 58.470469 58.470469 58.470469 58.470469 58.470469 58.470469 58.470469 58.470469 58.470469 

The final result i

Alhough the above works, it is not the most efficient way to calculate this integral.

You will notice that the integrand is an oscilliatory function.  Determine the oscillation frequency of the integrand (this you can do analytically!), the number of complete oscillations of the integrand, and the remaining extra part of an incomplete period at the end of the integrals range.  Print these results. [*Note:* Here it is useful to employ `//` for integer division (or the `int()` function)  to get the integer part of a calculation and `%` for modular arithmetic.]

In [4]:
# sin^2(x) has a period of pi
Nperiod = 48//np.pi
phi_extra = 48 - Nperiod*np.pi
# The following is an equivalent approach
# phi_extra = 48%np.pi
print("Period is pi:", np.pi)
print("Number of periods =", Nperiod)
print("Extra part of a period =", phi_extra)

Period is pi: 3.141592653589793
Number of periods = 15.0
Extra part of a period = 0.8761101961531068


It is more efficient to break up the complete integral into two: one integral over a period multiplied by the number of periods, and one integral over the remaining part of a period. Use Romberg integration to separately evaluate these two integrals and combine them to get the complete result.  Print the value of the total integral, its fractional error, and the total number of function evaluations required in this case.

In [5]:
### BEGIN SOLUTION
print("\nIntegration over one period:")
res1 = integ.romberg(f, 0, np.pi, show=True)
print("\nIntegration over remaining part of a period:")
res2 = integ.romberg(f, 0, phi_extra, show=True)
res = Nperiod*res1+res2
# We need to fill in the number of function evaluations by hand.
print("\nIntegral =", res, "with fractional error", np.abs(1-res/true_value), "from", 129+17, "function evaluations")
### END SOLUTION


Integration over one period:
Romberg integration of <function vectorize1.<locals>.vfunc at 0x00000259A6FD9B70> from [0, 3.141592653589793]

 Steps  StepSize   Results
     1  3.141593  4.442883 
     2  1.570796  3.792238  3.575356 
     4  0.785398  3.819944  3.829179  3.846100 
     8  0.392699  3.820198  3.820282  3.819689  3.819270 
    16  0.196350  3.820198  3.820198  3.820192  3.820200  3.820204 
    32  0.098175  3.820198  3.820198  3.820198  3.820198  3.820198  3.820198 
    64  0.049087  3.820198  3.820198  3.820198  3.820198  3.820198  3.820198  3.820198 
   128  0.024544  3.820198  3.820198  3.820198  3.820198  3.820198  3.820198  3.820198  3.820198 

The final result is 3.820197789028041 after 129 function evaluations.

Integration over remaining part of a period:
Romberg integration of <function vectorize1.<locals>.vfunc at 0x00000259A6FEC268> from [0, 0.8761101961531068]

 Steps  StepSize   Results
     1  0.876110  1.139625 
     2  0.438055  1.160793  1.167849 
     4

You should find that far fewer function evaluations are required when we split the integral as compared to integrating over the full range.  When integrating over the full interval, it is this interval that gets subdivided, which means we are "recalculating" the integral over one period repeatedly, all with different levels of accuracy.  Thus a lot of work goes into recalculating the same thing, which is clearly not needed.

**Optional exercise:** Note that we could do even better.  By symmetry, as seen in your plot above, we could instead integrate over a half period instead of a full period.  Feel free to repeat the above now using a half period.  You should find even fewer function evaluations are required! This is not required. But will be included in the solutions.

In [6]:
### BEGIN SOLUTION
# Though not necessary, here are the results
# cos^2(x) has a period of pi, so ...
Nhalfperiod = 48//np.pi*2
phi_extra = 48%(np.pi/2)
print("Number of half periods =", Nhalfperiod)
print("Extra part of a half period =", phi_extra)
print("\nIntegration over one half period:")
res1 = integ.romberg(f, 0, np.pi/2, show=True)
print("\nIntegration over remaining part of a half period:")
res2 = integ.romberg(f, 0, phi_extra, show=True)
res = Nhalfperiod*res1+res2
# We need to fill in the number of function evaluations by hand.
print("\nIntegral =", res, "with fractional error", np.abs(1-res/true_value), "from", 65+17, "function evaluations")
### END SOLUTION

Number of half periods = 30.0
Extra part of a half period = 0.8761101961531033

Integration over one half period:
Romberg integration of <function vectorize1.<locals>.vfunc at 0x00000259A6FE01E0> from [0, 1.5707963267948966]

 Steps  StepSize   Results
     1  1.570796  1.896119 
     2  0.785398  1.909972  1.914589 
     4  0.392699  1.910099  1.910141  1.909845 
     8  0.196350  1.910099  1.910099  1.910096  1.910100 
    16  0.098175  1.910099  1.910099  1.910099  1.910099  1.910099 
    32  0.049087  1.910099  1.910099  1.910099  1.910099  1.910099  1.910099 
    64  0.024544  1.910099  1.910099  1.910099  1.910099  1.910099  1.910099  1.910099 

The final result is 1.9100988945140078 after 65 function evaluations.

Integration over remaining part of a half period:
Romberg integration of <function vectorize1.<locals>.vfunc at 0x00000259A6FE0730> from [0, 0.8761101961531033]

 Steps  StepSize   Results
     1  0.876110  1.139625 
     2  0.438055  1.160793  1.167849 
     4  0.2190

## The Power of `quad`

As noted in class, `scipy.integrate.quad` is an extremely powerful integration routine.  It can do so many things that it has its own function, `scipy.integrate.quad_explain`, to explain some of its more esoteric features.  Here we will study one set of features it provides without us even asking for it: the ability to handle divergences in the integrand at the end points of the integration range and integration over (semi-)infinite intervals.

### Divergent Integrand

Consider the integral
$$ \int_0^1 \frac{\mathrm{d}x}{\sqrt{x}}. $$
You can perform this integral analytically.  Do so! We will use the result below.  

As a first numerical test, try to perform this integral using Romberg integration and see what happens.

In [7]:
### BEGIN SOLUTOIN
def f(x) : return 1/np.sqrt(x)
integ.romberg(f, 0, 1)
### END SOLUTION

C:\Users\James\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  
C:\Users\James\Anaconda3\lib\site-packages\scipy\integrate\quadrature.py:619: RuntimeWarning: invalid value encountered in double_scalars
  return (tmp * c - b)/(tmp - 1.0)
C:\Users\James\Anaconda3\lib\site-packages\scipy\integrate\quadrature.py:752: AccuracyWarning: divmax (10) exceeded. Latest difference = nan
  AccuracyWarning)


nan

You should have found that Romberg integration cannot handle this integral since it tries to evaluate the integrand at $x=0$, which is clearly a problem.  Now do the same with `quad`.  Print the value of the integral and the fractional error in the result. You should find it handles the integral with no problem and produces a very accurate result!

In [8]:
### BEGIN SOLUTION
(res, err) = integ.quad(f, 0, 1)
# The analytic value for the integral is 2. We use this in the
# fractional error calculation.
print("Integral =", res, " fractional error =", np.abs(1-res/2))
### END SOLUTION

Integral = 2.0000000000000004  fractional error = 2.220446049250313e-16


### Semi-infinite Integral Range

Consider the integral

$$ \int_1^\infty \frac{\mathrm{d}x}{x^2} . $$

Once again we can perform this integral analytically.  Clearly this integral **cannot be performed** using a method like Romberg integration, since we cannot evaluate them over an infinite range.  Similar to the previous case, if we needed to perform these problematic integrals using those techniques we would need to transform the integral in some way.  For this case it is relatively easy to do.  If we let $y\equiv 1/x$ then we can show

$$ \int_1^\infty \frac{\mathrm{d}x}{x^2} = \int_0^1 \mathrm{d}y, $$

a very simple integral to evaluate!

This was a simple case, other integrands may be much more tedious to transform.  Once again `quad` can handle the semi-infinite range for us.  Use `quad` to perform the integral in its original, semi-infinite form.  Print the value of the integral and the fractional error in this value. You should find that `quad` does exceptionally well, almost as if it were using the transformation we just discussed, .... [*Note:* We can specify infinity in a few ways.  Here I am using `np.inf`, though there exists `np.infty`, `scipy.inf`, `scipy.infty`, and probably many other definitions of the same thing.  They are all equivalent and any of them can be used.]

In [10]:
### BEGIN SOLUTION
def f(x) : return 1/x**2
(res, err) = integ.quad(f, 1, np.inf)
print("Integral =", res, " fractional error =", np.abs(1-res))
### END SOLUTION

Integral = 1.0  fractional error = 0.0


### Infinite Integral Range

Finally, `quad` can also handle a completely infinite range.  Consider the normalization integral for the Gaussian distribution that we have seen in previous assignments,

$$ \frac1{\sqrt{2\pi}\sigma} \int_{-\infty}^\infty \exp\left[ -\frac{(x-\mu)^2}{2\sigma^2} \right] \mathrm{d}x = 1. $$

Again we could transform this to integrals over finite ranges.  In this case we could use various techniques, shift the integral to remove the mean, $\mu$, from the integrand.  Then use symmetry to turn it into an integral over the semi-infinite range $(0,\infty)$.  Next split this integral into two pieces, an integral from $0$ to $1$ and an integral from $1$ to $\infty$.  Finally, this second integral can be handled by the transformation discussed in the semi-infinite case above.  Or, just let `quad` do it for us!

Calculate this integral using `quad`.  Print its value and fractional error.  Note that this result holds for arbitrary mean, $\mu$, and variance, $\sigma$; for your own purposes you should try a few cases.  Once again you should find it works extremely well!

In [11]:
### BEGIN SOLUTION
def gaussian(x, mu=0.0, sigma=1.0) :
    return 1/np.sqrt(2*np.pi)/sigma * np.exp(-(x-mu)**2/(2*sigma**2))

# One example
(res, err) = integ.quad(gaussian, -np.inf, np.inf, args=(-10., 30.))
print("Integral =", res, " fractional error =", np.abs(1-res))
### END SOLUTION

Integral = 1.0  fractional error = 0.0
